# Amazon Books – Data Preparation Pipeline

This notebook builds a complete, reproducible preprocessing pipeline for the Amazon Books dataset.
Starting from the raw Kaggle export, it produces:

1. A cleaned and deduplicated user–item interaction table (`ratings.csv`) with normalized timestamps and one canonical `itemId` per book title.
2. A time-aware leave-one-out (LOO) split with train, validation, and test targets, plus aligned user/item index maps.
3. Labeled interaction logs for training recommendation baselines and LLM prompts, including positive/negative labels.
4. Item–item similarity candidates and 100-user candidate pools with ground-truth targets injected.
5. Truncated user histories with at most 5 recent positives per user to control sequence length.
6. Final CSV exports of splits, candidate pools, and user/item maps under `csv_export/`.
7. Lightweight item-level metadata (title + one review field) and a richer per-item description table built from multiple reviews.
8. Consistency and quality checks on both interactions and metadata to ensure that all downstream components see a clean, one-to-one mapping between IDs and titles.

## 0. Reset local output folders

I start by clearing any previous outputs under `splits/`, `candidates_subset100/`, and `csv_export/`.
This ensures the notebook can be rerun from scratch without leftover Parquet or CSV files that might be inconsistent with the current code.

In [1]:
import os
import shutil
from pathlib import Path

BASE = Path(r"C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project")

folders_to_clean = [
    "splits",
    "candidates_subset100",
    "csv_export"
]

for d in folders_to_clean:
    p = BASE / d
    if p.exists():
        shutil.rmtree(p)
        print("[clean] removed", p)
    else:
        print("[clean] not found", p)

[clean] not found C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\splits
[clean] not found C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\candidates_subset100
[clean] not found C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\csv_export


## 1. Load and normalize raw Amazon ratings

The raw `Books_rating.csv` file from Kaggle is loaded and converted into a compact interaction table with four columns: `userId`, `itemId`, `rating`, and `timestamp`.
Timestamps are parsed into a consistent Unix second format, non-numeric ratings are coerced to floats, and only ratings in the closed interval `[1, 5]` are kept.
The result is saved as `ratings.csv`, which serves as the starting point for all subsequent filtering and deduplication.

In [2]:
import pandas as pd
import numpy as np

CSV_PATH = BASE / "Books_rating.csv"
assert CSV_PATH.exists(), f"Missing file: {CSV_PATH}"

df_raw = pd.read_csv(CSV_PATH)
print("Raw columns:", list(df_raw.columns))

df = pd.DataFrame({
    "userId":    df_raw["User_id"],
    "itemId":    df_raw["Id"],
    "rating":    pd.to_numeric(df_raw["review/score"], errors="coerce"),
    "timestamp": pd.to_datetime(df_raw["review/time"], errors="coerce"),
})

# Fill missing timestamps with a fixed dummy time and convert to Unix seconds
df["timestamp"] = df["timestamp"].fillna(pd.Timestamp(2000, 1, 1))
df["timestamp"] = df["timestamp"].astype("int64") // 10**9

# Keep only ratings in [1, 5]
df = df[(df["rating"] >= 1) & (df["rating"] <= 5)]

print(df.head(), df.dtypes)

ratings_csv = BASE / "ratings.csv"
df.to_csv(ratings_csv, index=False)
print(f"[saved] {ratings_csv} ~ {len(df):,} rows")

Raw columns: ['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness', 'review/score', 'review/time', 'review/summary', 'review/text']
           userId      itemId  rating  timestamp
0   AVCGYZL8FQQTD  1882931173     4.0          0
1  A30TK6U7DNS82R  0826414346     5.0          1
2  A3UH4UZ4RSVO82  0826414346     5.0          1
3  A2MVUWT453QH61  0826414346     4.0          1
4  A22X4XUPKF66MR  0826414346     4.0          1 userId        object
itemId        object
rating       float64
timestamp      int64
dtype: object
[saved] C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\ratings.csv ~ 3,000,000 rows


### 1.1 Apply user–item k-core filter

Next, I enforce a user–item k-core on `ratings.csv` so that every remaining user and item appears with at least 5 positive interactions.
The k-core procedure iteratively prunes users and items with fewer than the required interactions until all surviving entries satisfy the minimum-degree constraint.
This step removes extremely sparse users and rare items, stabilizing later splitting and similarity computations.

In [3]:
def kcore_filter(df, u_col="userId", i_col="itemId", k_user=5, k_item=5, max_iters=20, verbose=True):
    for it in range(max_iters):
        n0, u0, i0 = len(df), df[u_col].nunique(), df[i_col].nunique()
        uf = df[u_col].value_counts()
        vf = df[i_col].value_counts()

        df = df[df[u_col].isin(uf[uf >= k_user].index)]
        df = df[df[i_col].isin(vf[vf >= k_item].index)]

        n1, u1, i1 = len(df), df[u_col].nunique(), df[i_col].nunique()

        if verbose:
            print(f"[k-core {it+1}] rows {n0:,}->{n1:,}, users {u0:,}->{u1:,}, items {i0:,}->{i1:,}")

        if n1 == n0:
            break
    return df


# Load the normalized ratings and apply k-core
ratings_csv = BASE / "ratings.csv"
df = pd.read_csv(ratings_csv)

df = kcore_filter(df, k_user=5, k_item=5)
print(f"[after k-core] users={df['userId'].nunique():,}, items={df['itemId'].nunique():,}, rows={len(df):,}")

# Overwrite ratings.csv with the k-core filtered version
df.to_csv(ratings_csv, index=False)
print(f"[saved] {ratings_csv} ~ {len(df):,} rows")

[k-core 1] rows 3,000,000->1,077,091, users 1,008,972->82,519, items 221,998->69,986
[k-core 2] rows 1,077,091->977,301, users 82,519->77,225, items 69,986->29,668
[k-core 3] rows 977,301->951,522, users 77,225->70,381, items 29,668->28,742
[k-core 4] rows 951,522->944,212, users 70,381->70,121, items 28,742->27,027
[k-core 5] rows 944,212->941,724, users 70,121->69,534, items 27,027->26,952
[k-core 6] rows 941,724->940,863, users 69,534->69,496, items 26,952->26,770
[k-core 7] rows 940,863->940,525, users 69,496->69,417, items 26,770->26,757
[k-core 8] rows 940,525->940,434, users 69,417->69,416, items 26,757->26,731
[k-core 9] rows 940,434->940,416, users 69,416->69,411, items 26,731->26,731
[k-core 10] rows 940,416->940,416, users 69,411->69,411, items 26,731->26,731
[after k-core] users=69,411, items=26,731, rows=940,416
[saved] C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\ratings.csv ~ 940,416 rows


### 1.2 Enforce one canonical `itemId` per title

To ensure that each indexed item corresponds to a single logical book, I collapse multiple raw Amazon IDs that share the same (fuzzily normalized) title into one canonical `itemId`.

The steps are:
1. Normalize titles using lowercasing and simple text cleaning rules to group visually similar variants.
2. Build a mapping from normalized title to one canonical `itemId` (the smallest ID per title).
3. Remap all interactions to the canonical IDs and drop any rows that cannot be matched.
4. Remove duplicate `(userId, itemId, timestamp)` rows introduced by merging multiple raw IDs into one.

The resulting `ratings.csv` has a one-to-one mapping between canonical `itemId` and normalized book titles.

In [2]:
from pathlib import Path
import pandas as pd
import re

BASE = Path(".")
RAW_CSV_PATH = BASE / "Books_rating.csv"
RATINGS_PATH = BASE / "ratings.csv"

def normalize_title_for_dedup(title: str) -> str:
    if not isinstance(title, str):
        return ""
    t = title.lower()

    t = re.sub(r"\([^)]*\)", " ", t)

    t = re.sub(r"[-–—/:;]", " ", t)

    t = re.sub(r"[^a-z0-9\s]", " ", t)

    t = re.sub(r"\s+", " ", t).strip()

    STOPWORDS = {
        "the", "a", "an", "or", "and",
        "box", "boxed", "set", "edition", "ed",
        "volume", "vol", "volumes",
        "collection", "classic", "classics", "trilogy",
        "audio", "audiobook", "audiobooks", "recordings",
        "penguin", "bantam", "pocket", "enriched", "brilliance",
        "one", "voice"
    }
    tokens = [w for w in t.split() if w not in STOPWORDS]
    return " ".join(tokens)

assert RAW_CSV_PATH.exists(), f"Missing raw ratings file: {RAW_CSV_PATH}"
assert RATINGS_PATH.exists(), f"Missing k-core ratings file: {RATINGS_PATH}"

ratings = pd.read_csv(RATINGS_PATH)
print("Before title & interaction dedup:")
print(" users :", ratings["userId"].nunique())
print(" items :", ratings["itemId"].nunique())
print(" rows  :", len(ratings))

raw = pd.read_csv(RAW_CSV_PATH, usecols=["Id", "Title"])
raw = raw.rename(columns={"Id": "itemId", "Title": "title"})
raw = raw.dropna(subset=["title"])

raw["title_norm"] = raw["title"].map(normalize_title_for_dedup)

print("\nTitle stats in raw metadata:")
print(" Unique raw titles        :", raw["title"].nunique())
print(" Unique normalized (fuzzy):", raw["title_norm"].nunique())

raw_item = (
    raw
    .sort_values(["itemId", "title_norm"])
    .drop_duplicates(subset=["itemId"], keep="first")
)

title_to_canon = (
    raw_item
    .sort_values(["title_norm", "itemId"])
    .drop_duplicates(subset=["title_norm"], keep="first")
    .set_index("title_norm")["itemId"]
)

print(" Unique normalized titles in mapping:", title_to_canon.shape[0])

id_to_title_norm = raw_item.set_index("itemId")["title_norm"]

id_to_canon = id_to_title_norm.map(title_to_canon)

ratings["title_norm"] = ratings["itemId"].map(id_to_title_norm)
missing_title_norm = ratings["title_norm"].isna().sum()
print("\nRows in ratings missing title_norm from metadata:", missing_title_norm)
ratings = ratings.dropna(subset=["title_norm"])

ratings["canonical_itemId"] = ratings["itemId"].map(id_to_canon)
missing_canon = ratings["canonical_itemId"].isna().sum()
print("Rows in ratings missing canonical_itemId (should be 0 or very small):", missing_canon)
ratings = ratings.dropna(subset=["canonical_itemId"])

ratings["itemId"] = ratings["canonical_itemId"]

ratings = ratings.drop(columns=["canonical_itemId", "title_norm"])

ratings = (
    ratings
    .sort_values(["userId", "itemId", "timestamp"])
    .drop_duplicates(subset=["userId", "itemId"], keep="first")
    .reset_index(drop=True)
)

print("\nAfter FUZZY title + interaction dedup:")
print(" users :", ratings["userId"].nunique())
print(" items :", ratings["itemId"].nunique())
print(" rows  :", len(ratings))

dup_interactions = ratings.duplicated(subset=["userId", "itemId", "timestamp"]).sum()
print("Duplicate interactions remaining (should be 0):", dup_interactions)

ratings.to_csv(RATINGS_PATH, index=False)
print(f"\nSaved deduplicated ratings with fuzzy titles to {RATINGS_PATH}")

Before title & interaction dedup:
 users : 69411
 items : 20500
 rows  : 495535

Title stats in raw metadata:
 Unique raw titles        : 212403
 Unique normalized (fuzzy): 198553
 Unique normalized titles in mapping: 198553

Rows in ratings missing title_norm from metadata: 0
Rows in ratings missing canonical_itemId (should be 0 or very small): 0

After FUZZY title + interaction dedup:
 users : 69411
 items : 20500
 rows  : 495535
Duplicate interactions remaining (should be 0): 0

Saved deduplicated ratings with fuzzy titles to ratings.csv


### 1.3 Deduplication validation and metadata coverage

After fuzzy title deduplication, I run several sanity checks to confirm that the interaction table and the raw metadata stay consistent:

1. Count how many normalized titles still map to multiple raw `itemId` values, and verify that each normalized title corresponds to exactly one canonical ID.
2. Check that there are no remaining duplicate `(userId, itemId, timestamp)` rows after collapsing IDs.
3. Confirm that every `itemId` appearing in `ratings.csv` has a corresponding entry in the raw metadata file.

These checks provide evidence that the fuzzy title merge did not introduce conflicts or orphan items.

In [3]:
print("\nDEDUP VALIDATION\n")

tn_to_n_ids = (
    raw_item
    .groupby("title_norm")["itemId"]
    .nunique()
)
multi_id_norms = (tn_to_n_ids > 1).sum()
print("title_norm groups with MULTIPLE raw itemIds in raw_item (EXPECTED, shows fuzzy-merged groups):", int(multi_id_norms))

df_check = raw_item.copy()
df_check["canonical_itemId"] = df_check["title_norm"].map(title_to_canon)
n_canon_per_norm = (
    df_check.groupby("title_norm")["canonical_itemId"]
    .nunique()
)
bad_norms = (n_canon_per_norm > 1).sum()
print("title_norm groups mapping to >1 canonical_itemId (should be 0):",
      int(bad_norms))

dup_ui = ratings.duplicated(subset=["userId", "itemId"]).sum()
dup_uit = ratings.duplicated(subset=["userId", "itemId", "timestamp"]).sum()
print("Duplicate (userId, itemId) rows in ratings (should be 0):",
      int(dup_ui))
print("Duplicate (userId, itemId, timestamp) rows in ratings (should be 0):",
      int(dup_uit))

missing_meta = (~ratings["itemId"].isin(raw_item["itemId"])).sum()
print("ItemIds in ratings WITHOUT metadata (should be 0):",
      int(missing_meta))

print("\nFinal stats after all dedup:")
print(" users :", ratings["userId"].nunique())
print(" items :", ratings["itemId"].nunique())
print(" rows  :", len(ratings))


DEDUP VALIDATION

title_norm groups with MULTIPLE raw itemIds in raw_item (EXPECTED, shows fuzzy-merged groups): 15588
title_norm groups mapping to >1 canonical_itemId (should be 0): 0
Duplicate (userId, itemId) rows in ratings (should be 0): 0
Duplicate (userId, itemId, timestamp) rows in ratings (should be 0): 0
ItemIds in ratings WITHOUT metadata (should be 0): 0

Final stats after all dedup:
 users : 69411
 items : 20500
 rows  : 495535


In [7]:
sample_norm = raw["title_norm"].sample(1).iloc[0]
print("title_norm =", sample_norm)

raw[raw["title_norm"] == sample_norm][["itemId", "title"]].head(50)

title_norm = 125 best gluten free recipes


,itemId,title
1027645,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027646,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027647,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027648,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027649,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027650,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027651,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027652,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027653,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES
1027654,B000N6KP5Q,125 BEST GLUTEN FREE RECIPES


In [13]:
item_meta = raw[["itemId", "title"]].drop_duplicates()

ratings_with_title = ratings.merge(item_meta, on="itemId", how="left")

dup_titles = (
    ratings_with_title
    .groupby("itemId")["title"]
    .nunique()
    .reset_index(name="n_titles")
)

print(
    "How many canonical_itemId still have >1 distinct raw titles:",
    (dup_titles["n_titles"] > 1).sum()
)

dup_titles[dup_titles["n_titles"] > 1].head(20)

How many canonical_itemId still have >1 distinct raw titles: 0


,itemId,n_titles


In [10]:
dup_full = ratings.groupby(["userId", "itemId", "timestamp"]).size()
dup_full[dup_full > 1].head()

Series([], dtype: int64)

In [11]:
missing_meta = set(ratings["itemId"]) - set(raw["itemId"])
print("ItemIds missing metadata:", len(missing_meta))

ItemIds missing metadata: 0


### 1.4 Manual inspection of merged title groups

To visually confirm the quality of the fuzzy title merge, I randomly sample a few normalized titles and print:

1. All raw titles that map into each sampled normalized title.
2. The chosen canonical `itemId` for that group.
3. The number of distinct users and interactions that now refer to the canonical item.

These spot-checks help verify that obviously unrelated books are not being merged, and that the canonical ID indeed aggregates closely related variants of the same title.

In [4]:
import pandas as pd

raw = pd.read_csv(RAW_CSV_PATH, usecols=["Id", "Title"])
raw = raw.rename(columns={"Id": "itemId", "Title": "title"})
raw = raw.dropna(subset=["title"])

raw["title_norm"] = raw["title"].map(normalize_title_for_dedup)

print("Unique raw titles        :", raw["title"].nunique())
print("Unique normalized titles :", raw["title_norm"].nunique())
print()

canon_for_norm = (
    raw.sort_values(["title_norm", "itemId"])
       .drop_duplicates(subset="title_norm", keep="first")[["title_norm", "itemId"]]
       .rename(columns={"itemId": "canonical_itemId"})
)

norm_to_canon = dict(
    zip(canon_for_norm["title_norm"], canon_for_norm["canonical_itemId"])
)

N_SAMPLES = 5

sample_norms = (
    canon_for_norm["title_norm"]
    .dropna()
    .sample(N_SAMPLES, random_state=42)
    .tolist()
)
print("MANUAL DEDUP INSPECTION\n")
print(f"Sampled {N_SAMPLES} title_norm values:\n", sample_norms, "\n")

for tnorm in sample_norms:
    print(f"title_norm = {tnorm!r}")

    block_raw = raw[raw["title_norm"] == tnorm].sort_values("title")
    print("\nRaw titles for this title_norm (before dedup):")
    print(
        block_raw[["itemId", "title"]]
        .drop_duplicates()
        .head(20)
        .to_string(index=False)
    )

    canon_id = norm_to_canon[tnorm]
    print(f"\nChosen canonical_itemId : {canon_id}")

    block_ratings = ratings[ratings["itemId"] == canon_id]

    n_users = block_ratings["userId"].nunique()
    n_rows  = len(block_ratings)

    print(
        f"In cleaned ratings: {n_rows} interactions, "
        f"{n_users} unique users, itemId = {canon_id}"
    )
    print()

print("\nDone. For each sampled title_norm, should see：")
print("There are pbbly a lot of similar titles merged together in raw")
print("There is only one itemId（canonical_itemId）kept in cleaned ratings")

Unique raw titles        : 212403
Unique normalized titles : 198553

MANUAL DEDUP INSPECTION

Sampled 5 title_norm values:
 ['shoot everything you ever wanted to know about 35mm photography', 'edmund s used cars trucks prices ratings 1999 fall u3303', 'epic role playing rules manual', 'pictorial tribute to crewe works in age of steam', 'mexican cooking at academy at academy'] 

title_norm = 'shoot everything you ever wanted to know about 35mm photography'

Raw titles for this title_norm (before dedup):
    itemId                                                             title
0817458697 Shoot!: Everything you Ever Wanted to Know About 35Mm Photography

Chosen canonical_itemId : 0817458697
In cleaned ratings: 0 interactions, 0 unique users, itemId = 0817458697

title_norm = 'edmund s used cars trucks prices ratings 1999 fall u3303'

Raw titles for this title_norm (before dedup):
    itemId                                                                            title
087759645X Edmu

### 1.5 Interaction statistics and positive/negative thresholds

Before splitting the data, I analyze how many interactions each user has and how different rating thresholds affect the number of “positive” events.

For several candidate thresholds (2.0, 3.0, 4.0, 5.0), I compute:
1. How many users would retain at least a given number of positives.
2. The total counts of positive versus negative interactions.

This exploration motivates the later choice of a high positive threshold (`rating ≥ 5`) and a minimum number of positives per user when constructing training and evaluation splits.

In [15]:
import pandas as pd

# Load final deduplicated ratings BEFORE LOO split
ratings = pd.read_csv(BASE / "ratings.csv")

print("Total users in ratings.csv:", ratings["userId"].nunique())
print("Total interactions:", len(ratings))

# Compute number of interactions per user once (for later use)
user_inter_count = ratings.groupby("userId").size()

print("\nUsers with >=4 total interactions:", (user_inter_count >= 4).sum())
print("Users with >=5 total interactions:", (user_inter_count >= 5).sum())
print("Users with >=10 total interactions:", (user_inter_count >= 10).sum())

# Count positive/negative interactions under different thresholds
for th in [2.0, 3.0, 4.0,5.0]:
    print(f" Threshold = {th}")

    # Positive and negative definitions
    pos = ratings[ratings["rating"] >= th]
    neg = ratings[ratings["rating"] < th]

    # Per-user positive counts
    cnt_pos = pos.groupby("userId").size()

    print("Users with >=2 positives:", (cnt_pos >= 2).sum())
    print("Users with >=3 positives:", (cnt_pos >= 3).sum())
    print("Users with >=5 positives:", (cnt_pos >= 5).sum())
    print("Total users considered (have >=1 positive):", cnt_pos.size)

    print("Total positive interactions:", len(pos))
    print("Total negative interactions:", len(neg))

print("\nDone.")

Total users in ratings.csv: 69411
Total interactions: 495535

Users with >=4 total interactions: 38336
Users with >=5 total interactions: 28092
Users with >=10 total interactions: 9824
 Threshold = 2.0
Users with >=2 positives: 59887
Users with >=3 positives: 49107
Users with >=5 positives: 26232
Total users considered (have >=1 positive): 67573
Total positive interactions: 474616
Total negative interactions: 20919
 Threshold = 3.0
Users with >=2 positives: 58206
Users with >=3 positives: 47197
Users with >=5 positives: 24398
Total users considered (have >=1 positive): 66302
Total positive interactions: 449362
Total negative interactions: 46173
 Threshold = 4.0
Users with >=2 positives: 54883
Users with >=3 positives: 43486
Users with >=5 positives: 21016
Total users considered (have >=1 positive): 63823
Total positive interactions: 393215
Total negative interactions: 102320
 Threshold = 5.0
Users with >=2 positives: 44822
Users with >=3 positives: 33538
Users with >=5 positives: 14064

## 2. Create time-aware leave-one-out (LOO) split

Using the cleaned `ratings.csv`, I build a time-aware leave-one-out split:

1. Convert timestamps into a consistent datetime series, automatically detecting whether the input is in seconds or milliseconds.
2. Keep only interactions with `rating ≥ 5.0` as positives.
3. For each user with at least 5 positives, sort interactions by time and label:
   1. the most recent item as the **test** target,
   2. the second most recent as the **validation** target,
   3. all earlier positives as **train** history.
4. Save train histories and val/test targets as Parquet files under `splits/`, along with contiguous user and item index maps.

The resulting splits preserve temporal order and avoid information leakage from future interactions.

In [16]:
import pandas as pd
import numpy as np
from pathlib import Path

def _detect_ts_unit(ts_series: pd.Series) -> str:
    vmax = float(ts_series.max())
    # Heuristic: if timestamps are very large, assume milliseconds
    return "ms" if vmax > 1e12 else "s"

def time_aware_loo_split(
    ratings_csv: str,
    out_dir: str,
    rating_threshold: float = 5.0,
    min_positives: int = 5,
    also_csv: bool = False,
):
    out = Path(out_dir)
    (out / "splits").mkdir(parents=True, exist_ok=True)

    ratings = pd.read_csv(ratings_csv)

    need = {"userId", "itemId", "rating", "timestamp"}
    missing = need - set(ratings.columns)
    if missing:
        raise ValueError(f"ratings.csv missing columns: {missing}")

    # Convert timestamp column to pandas datetime with auto unit detection
    try:
        unit = _detect_ts_unit(ratings["timestamp"])
        ratings["ts"] = pd.to_datetime(ratings["timestamp"], unit=unit)
    except Exception:
        # Fallback: let pandas guess
        ratings["ts"] = pd.to_datetime(ratings["timestamp"], unit="s", origin="unix", errors="ignore")

    # Keep only positive interactions
    pos = ratings[ratings["rating"] >= rating_threshold].copy()

    # Sort by (userId, timestamp)
    pos = pos.sort_values(["userId", "ts"], kind="mergesort").drop_duplicates(
        ["userId", "itemId"], keep="first"
    )

    # Filter users with at least `min_positives` positives
    cnt = pos.groupby("userId")["itemId"].transform("size")
    pos = pos[cnt >= min_positives].copy()

    # Sort again (just to be safe after filtering)
    pos = pos.sort_values(["userId", "ts"], kind="mergesort")

    # Index within each user
    pos["n"] = pos.groupby("userId")["userId"].transform("size")
    pos["idx"] = pos.groupby("userId").cumcount()

    pos["split"] = "train"
    pos.loc[pos["idx"] == pos["n"] - 1, "split"] = "test"
    pos.loc[pos["idx"] == pos["n"] - 2, "split"] = "val"

    train = pos[pos["split"] == "train"][["userId", "itemId", "ts"]].reset_index(drop=True)
    val_targets = pos[pos["split"] == "val"][["userId", "itemId", "ts"]].reset_index(drop=True)
    test_targets = pos[pos["split"] == "test"][["userId", "itemId", "ts"]].reset_index(drop=True)

    # Build contiguous user/item ID maps
    uids = pd.DataFrame(sorted(train["userId"].unique()), columns=["userId"])
    uids["uid"] = range(len(uids))

    iids = pd.DataFrame(sorted(train["itemId"].unique()), columns=["itemId"])
    iids["iid"] = range(len(iids))

    # Map val/test userId/itemId into the same index space
    val_idx = (
        val_targets.merge(uids, on="userId", how="inner")
        .merge(iids, on="itemId", how="left")
        .drop(columns=["itemId"])
    )

    test_idx = (
        test_targets.merge(uids, on="userId", how="inner")
        .merge(iids, on="itemId", how="left")
        .drop(columns=["itemId"])
    )

    sp = out / "splits"
    train.to_parquet(sp / "train.parquet", index=False)
    if len(val_targets):
        val_targets.to_parquet(sp / "val_targets.parquet", index=False)
        val_idx.to_parquet(sp / "val_targets_indexed.parquet", index=False)
    test_targets.to_parquet(sp / "test_targets.parquet", index=False)
    test_idx.to_parquet(sp / "test_targets_indexed.parquet", index=False)

    uids.to_parquet(sp / "user_id_map.parquet", index=False)
    iids.to_parquet(sp / "item_id_map.parquet", index=False)

    # Save train with indexed ids
    train_idx = (
        train.merge(uids, on="userId", how="inner")
        .merge(iids, on="itemId", how="inner")
        .drop(columns=["userId", "itemId"])
    )
    train_idx.to_parquet(sp / "train_indexed.parquet", index=False)

    if also_csv:
        for p in [
            "train",
            "val_targets",
            "test_targets",
            "user_id_map",
            "item_id_map",
            "train_indexed",
            "val_targets_indexed",
            "test_targets_indexed",
        ]:
            df = pd.read_parquet(sp / f"{p}.parquet")
            df.to_csv(sp / f"{p}.csv", index=False)

    cold_val = int(val_idx["iid"].isna().sum()) if len(val_idx) else 0
    cold_test = int(test_idx["iid"].isna().sum()) if len(test_idx) else 0

    stats = f"""Time-aware LOO split summary
Users (TRAIN map): {len(uids)}
Items (TRAIN map): {len(iids)}
TRAIN positives   : {len(train)}
VAL users          : {val_idx["uid"].nunique() if len(val_idx) else 0}
TEST users         : {test_idx["uid"].nunique() if len(test_idx) else 0}
Cold-start VAL items : {cold_val}
Cold-start TEST items: {cold_test}
"""
    (sp / "stats.txt").write_text(stats, encoding="utf-8")
    print(stats)

In [17]:
time_aware_loo_split(
    ratings_csv=str(BASE / "ratings.csv"),
    out_dir=str(BASE),
    rating_threshold=5.0,
    min_positives=5,
    also_csv=False,
)

Time-aware LOO split summary
Users (TRAIN map): 14064
Items (TRAIN map): 18783
TRAIN positives   : 141530
VAL users          : 14064
TEST users         : 14064
Cold-start VAL items : 477
Cold-start TEST items: 1146



### 2.1 Ensure validation and test items are seen in training

After creating the time-aware splits, I filter out any validation or test targets whose items never appear in the training histories.
Only users whose target items are included in the training item set are kept, and the filtered val/test target files overwrite the originals.

This step removes cold-start items from evaluation and guarantees that every evaluated target item has been observed at least once in the training data.

In [18]:
import pandas as pd

SPLITS = BASE / "splits"
train_idx = pd.read_parquet(SPLITS/"train_indexed.parquet")
val_idx   = pd.read_parquet(SPLITS/"val_targets_indexed.parquet")
test_idx  = pd.read_parquet(SPLITS/"test_targets_indexed.parquet")

train_items = set(train_idx["iid"].unique())
val_keep  = val_idx[val_idx["iid"].isin(train_items)].copy()
test_keep = test_idx[test_idx["iid"].isin(train_items)].copy()

val_keep.to_parquet(SPLITS/"val_targets_indexed.parquet", index=False)
test_keep.to_parquet(SPLITS/"test_targets_indexed.parquet", index=False)
print("[covered] kept val:", len(val_keep), " / test:", len(test_keep))


[covered] kept val: 13587  / test: 12918


### 2.2 Build labeled interactions and training-only history

Using the full interaction table, I:

1. Assign a binary label `pos_neg_label` based on `rating ≥ 5` (positive) versus `< 5` (negative).
2. Tag all validation and test target `(userId, itemId)` pairs and mark them with an `is_target` flag.
3. Save a global `interactions_lookup.csv` containing every labeled interaction plus the target flag.
4. Remove val/test target rows to produce `interactions_for_training.csv`, which contains only non-target interactions.

I also check per-user statistics to ensure most users have at least 5 interactions and at least one positive and one negative for training.

In [19]:
import pandas as pd
from pathlib import Path

BASE = Path(BASE)

ratings_path = BASE / "ratings.csv"
ratings = pd.read_csv(ratings_path)

print("Loaded ratings.csv")
print("  users :", ratings["userId"].nunique())
print("  items :", ratings["itemId"].nunique())
print("  rows  :", len(ratings))

THRESHOLD = 5  # rating == 5 -> positive, <=4 -> negative
ratings["pos_neg_label"] = (ratings["rating"] >= THRESHOLD).astype(int)

print("\nOverall label stats (before removing targets):")
print("  positives:", (ratings["pos_neg_label"] == 1).sum())
print("  negatives:", (ratings["pos_neg_label"] == 0).sum())

val_targets  = pd.read_parquet(BASE / "splits/val_targets.parquet")
test_targets = pd.read_parquet(BASE / "splits/test_targets.parquet")

val_pairs  = val_targets[["userId", "itemId"]].drop_duplicates()
test_pairs = test_targets[["userId", "itemId"]].drop_duplicates()

val_test_pairs = pd.concat([val_pairs, test_pairs], ignore_index=True).drop_duplicates()
val_test_pairs["is_target"] = 1

print("\nVal/test target pairs:")
print("  val targets :", len(val_pairs))
print("  test targets:", len(test_pairs))
print("  unique pairs:", len(val_test_pairs))

ratings = ratings.merge(val_test_pairs, on=["userId", "itemId"], how="left")
ratings["is_target"] = ratings["is_target"].fillna(0).astype(int)

print("Target flag in ratings:")
print("  rows with is_target=1:", (ratings["is_target"] == 1).sum())

lookup_path = BASE / "csv_export" / "interactions_lookup.csv"
lookup_path.parent.mkdir(parents=True, exist_ok=True)

ratings.to_csv(lookup_path, index=False)
print("\nSaved interactions_lookup.csv to:", lookup_path)

train_interactions = ratings[ratings["is_target"] == 0].copy()

print("\nAfter removing val/test targets:")
print("  users :", train_interactions["userId"].nunique())
print("  items :", train_interactions["itemId"].nunique())
print("  rows  :", len(train_interactions))

cnt_total = train_interactions.groupby("userId").size()
cnt_pos = train_interactions[train_interactions["pos_neg_label"] == 1].groupby("userId").size()
cnt_neg = train_interactions[train_interactions["pos_neg_label"] == 0].groupby("userId").size()

valid_users = [
    u for u in cnt_total.index
    if cnt_total.get(u, 0) >= 5
       and cnt_pos.get(u, 0) >= 1
       and cnt_neg.get(u, 0) >= 1
]
valid_users = set(valid_users)

print("\nUsers satisfying >=5 interactions and at least 1 pos & 1 neg:")
print("  count:", len(valid_users))

val_users = set(val_pairs["userId"].unique())
test_users = set(test_pairs["userId"].unique())
val_test_users = val_users | test_users

missing_users = val_test_users - valid_users
if missing_users:
    print("\nWARNING: some val/test users do not meet >=5 & pos/neg condition.")
    print("  number of such users:", len(missing_users))
    # Still include them so that LLM has history
    valid_users |= missing_users
else:
    print("\nAll val/test users already satisfy the condition.")

# Apply final user filter
train_interactions = train_interactions[train_interactions["userId"].isin(valid_users)].copy()

print("\nFinal training interactions stats:")
print("  users :", train_interactions["userId"].nunique())
print("  items :", train_interactions["itemId"].nunique())
print("  rows  :", len(train_interactions))

train_path = BASE / "csv_export" / "interactions_for_training.csv"
train_interactions.to_csv(train_path, index=False)
print("\nSaved interactions_for_training.csv to:", train_path)

Loaded ratings.csv
  users : 69411
  items : 20500
  rows  : 495535

Overall label stats (before removing targets):
  positives: 269476
  negatives: 226059

Val/test target pairs:
  val targets : 14064
  test targets: 14064
  unique pairs: 28128
Target flag in ratings:
  rows with is_target=1: 28128

Saved interactions_lookup.csv to: csv_export\interactions_lookup.csv

After removing val/test targets:
  users : 69411
  items : 20498
  rows  : 467407

Users satisfying >=5 interactions and at least 1 pos & 1 neg:
  count: 21818

  number of such users: 4360

Final training interactions stats:
  users : 26178
  items : 20435
  rows  : 345571

Saved interactions_for_training.csv to: csv_export\interactions_for_training.csv


## 3. Build implicit feedback matrix and item–item baseline

From the indexed train split, I construct a sparse user–item implicit feedback matrix `R` based on the last 200 interactions per user.
By computing `C = RᵀR`, I obtain an approximate item–item co-occurrence similarity structure and derive, for each item, a small list of its most similar neighbors.

This `item_sim_map` can be used as a classical item–item recommendation baseline and later to build hybrid candidate pools that mix popularity and similarity-based candidates.

In [20]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.sparse import csr_matrix

SPLITS = BASE / "splits"
train_idx = pd.read_parquet(SPLITS/"train_indexed.parquet")   # [uid, iid, ts]
val_idx   = pd.read_parquet(SPLITS/"val_targets_indexed.parquet")   # [uid, val_item(iid), ts_val]
test_idx  = pd.read_parquet(SPLITS/"test_targets_indexed.parquet")  # [uid, test_item(iid), ts_test]

U = int(train_idx["uid"].max()) + 1
I = int(train_idx["iid"].max()) + 1

user_seen = train_idx.groupby("uid")["iid"].apply(set).to_dict()

def candidate_coverage(cand_df, targets_df, tgt_col="iid"):
    df = cand_df.merge(targets_df[["uid", tgt_col]], on="uid", how="inner")
    df = df[df[tgt_col].notna()]
    return np.mean([(int(t) in set(c)) for t, c in zip(df[tgt_col], df["candidates"])])

In [21]:
train_idx_sorted = train_idx.sort_values(["uid","ts"]).groupby("uid").tail(200)

R = csr_matrix(
    (np.ones(len(train_idx_sorted), dtype=np.float32),
     (train_idx_sorted["uid"].astype(int).values,
      train_idx_sorted["iid"].astype(int).values)),
    shape=(U, I),
    dtype=np.float32
)

C = (R.T @ R).tocsr()
C.setdiag(0); C.eliminate_zeros()

M_SIM = 50
item_sim_map = {}
for iid in range(I):
    a, b = C.indptr[iid], C.indptr[iid+1]
    if a == b:
        item_sim_map[iid] = []
        continue
    neigh = C.indices[a:b]
    vals  = C.data[a:b]
    if len(neigh) > M_SIM:
        top = np.argpartition(-vals, M_SIM)[:M_SIM]
        neigh, vals = neigh[top], vals[top]
    order = np.argsort(-vals)
    item_sim_map[iid] = neigh[order].tolist()

print(f"[item-sim] built for I={I:,}. Example of item 0:", item_sim_map.get(0, [])[:10])

[item-sim] built for I=18,783. Example of item 0: [7, 22, 26, 28, 77, 84, 202, 295, 477, 904]


### 3.1 Create 100-user candidate subset

To keep LLM-based experiments computationally manageable, I select 100 users who have both validation and test targets.
For these users, I build candidate lists by ranking items by global popularity and excluding items already seen in each user’s train history.
The resulting candidate pools, along with their val/test targets, are saved under `candidates_subset100/` for later use.

In [22]:
import pandas as pd
import numpy as np

SPLITS = BASE / "splits"

train_idx = pd.read_parquet(SPLITS / "train_indexed.parquet")
val_idx   = pd.read_parquet(SPLITS / "val_targets_indexed.parquet")
test_idx  = pd.read_parquet(SPLITS / "test_targets_indexed.parquet")

users_with_val  = set(val_idx['uid'].unique())
users_with_test = set(test_idx['uid'].unique())
candidate_users = sorted(users_with_val & users_with_test)

subset_users = candidate_users[:100]

print("Users with val+test:", len(candidate_users))
print("Selected 100 users:", len(subset_users))

train_sub = train_idx[train_idx['uid'].isin(subset_users)].copy()
val_tgt   = val_idx[val_idx['uid'].isin(subset_users)].copy()
test_tgt  = test_idx[test_idx['uid'].isin(subset_users)].copy()

item_pop_series = train_sub['iid'].value_counts().astype(float)
item_pop_series /= item_pop_series.max()
item_pop = item_pop_series.to_dict()
item_popular = sorted(item_pop.items(), key=lambda x: -x[1])

def build_pool_for_user(uid, k=50):
    seen = set(train_sub.loc[train_sub['uid'] == uid, 'iid'].astype(int))
    pool = [int(i) for i, _ in item_popular if i not in seen][:k]
    return pool

rows = []
for u in subset_users:
    rows.append({
        "uid": int(u),
        "candidates": build_pool_for_user(int(u), k=50)
    })

cand = pd.DataFrame(rows)

OUT = BASE / "candidates_subset100"
OUT.mkdir(parents=True, exist_ok=True)
print("Saving 100-user candidates to:", OUT)

cand.to_parquet(OUT / "val.parquet",  index=False)
cand.to_parquet(OUT / "test.parquet", index=False)

val_tgt.to_parquet(OUT / "val_targets_indexed.parquet",  index=False)
test_tgt.to_parquet(OUT / "test_targets_indexed.parquet", index=False)

print("✓ Saved 100-user subset to", OUT)

Users with val+test: 12542
Selected 100 users: 100
Saving 100-user candidates to: candidates_subset100
✓ Saved 100-user subset to candidates_subset100


## 4. Force-add ground-truth items into candidate pools

For the 100-user subset, I guarantee that each user’s validation and test target items are always included in their candidate lists.

For each user:
1. Load the initial popularity-based candidate list.
2. Compute the union of candidate items with that user’s val/test target items.
3. Write back updated candidate lists so that val and test now share identical candidate pools.

This ensures that evaluation metrics are well-defined for every user and that targets are never missing from the candidate set.

In [23]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project")
CANDS = BASE / "candidates_subset100"

cand_val  = pd.read_parquet(CANDS / "val.parquet")
cand_test = pd.read_parquet(CANDS / "test.parquet")

val_tgt   = pd.read_parquet(CANDS / "val_targets_indexed.parquet")
test_tgt  = pd.read_parquet(CANDS / "test_targets_indexed.parquet")

print("Loaded cand_val :", cand_val.shape)
print("Loaded cand_test:", cand_test.shape)
print("Loaded val_tgt  :", val_tgt.shape)
print("Loaded test_tgt :", test_tgt.shape)

cand_map = {int(row.uid): list(row.candidates) for _, row in cand_val.iterrows()}

for uid in sorted(cand_map.keys()):
    uid = int(uid)

    val_items  = set(val_tgt[val_tgt["uid"] == uid]["iid"].astype(int).tolist())
    test_items = set(test_tgt[test_tgt["uid"] == uid]["iid"].astype(int).tolist())
    gt_items   = val_items | test_items

    cur_list = cand_map[uid]
    cur_set  = set(int(i) for i in cur_list)

    added = 0
    for g in gt_items:
        g = int(g)
        if g not in cur_set:
            cur_list.append(g)
            cur_set.add(g)
            added += 1

    cand_map[uid] = cur_list

cand_fixed = pd.DataFrame(
    [{"uid": uid, "candidates": cand_map[uid]} for uid in sorted(cand_map.keys())]
)

cand_fixed.to_parquet(CANDS / "val.parquet", index=False)
cand_fixed.to_parquet(CANDS / "test.parquet", index=False)

print("✓ Ground-truth items have been forced into candidate pools.")
print("  New cand_val shape :", cand_fixed.shape)
print("  (val / test now share exactly the same candidate pools)")

Loaded cand_val : (100, 2)
Loaded cand_test: (100, 2)
Loaded val_tgt  : (100, 4)
Loaded test_tgt : (100, 4)
✓ Ground-truth items have been forced into candidate pools.
  New cand_val shape : (100, 2)
  (val / test now share exactly the same candidate pools)


### 4.1 Build labeled candidate pools

Using the candidate lists, the user/item index maps, and the global interaction lookup table, I construct final labeled candidate pools:

1. Flatten the per-user candidate lists into a long table of `(uid, iid)` pairs.
2. Join with the interaction labels to assign `label` (positive/negative) for each candidate when available.
3. Mark which rows correspond to ground-truth targets via an `is_target` flag.
4. Export separate CSV files `candidate_pool_val.csv` and `candidate_pool_test.csv` under the project root.

These files are the main inputs for both traditional recommenders and LLM-based rerankers.

In [24]:
import pandas as pd
from pathlib import Path

BASE = Path(BASE)
SPLITS = BASE / "splits"
CANDS = BASE / "candidates_subset100"
LOOKUP = BASE / "csv_export" / "interactions_lookup.csv"

user_map = pd.read_parquet(SPLITS / "user_id_map.parquet")
item_map = pd.read_parquet(SPLITS / "item_id_map.parquet")

print("User map:", user_map.shape)
print("Item map:", item_map.shape)

lookup = pd.read_csv(LOOKUP)
print("Lookup table:", lookup.shape)

lookup_uid = lookup.merge(user_map, on="userId", how="left")
lookup_uid_iid = lookup_uid.merge(item_map, on="itemId", how="left")

lookup_uid_iid = lookup_uid_iid[["uid", "iid", "pos_neg_label"]].dropna()
lookup_uid_iid["uid"] = lookup_uid_iid["uid"].astype(int)
lookup_uid_iid["iid"] = lookup_uid_iid["iid"].astype(int)

lookup_uid_iid.set_index(["uid", "iid"], inplace=True)


def build_candidate_pool(split: str):
    print(f"\n--- Building candidate pool for {split} ---")

    cand_path = CANDS / f"{split}.parquet"
    print("Loading candidates from:", cand_path)
    candidates = pd.read_parquet(cand_path)
    print("Candidates shape (raw):", candidates.shape)
    print("Candidate columns:", list(candidates.columns))

    rows = []
    for _, row in candidates.iterrows():
        uid = int(row["uid"])
        for iid in row["candidates"]:
            rows.append((uid, int(iid)))

    cand_flat = pd.DataFrame(rows, columns=["uid", "iid"])
    print("Flattened candidates shape:", cand_flat.shape)

    tgt_path = SPLITS / f"{split}_targets_indexed.parquet"
    tgt = pd.read_parquet(tgt_path)[["userId", "iid"]].copy()

    tgt = tgt.merge(user_map, on="userId", how="left")[["uid", "iid"]]
    tgt["uid"] = tgt["uid"].astype(int)
    tgt["iid"] = tgt["iid"].astype(int)

    tgt["is_target"] = 1

    cand_with_tgt = pd.concat([cand_flat, tgt[["uid", "iid"]]], ignore_index=True).drop_duplicates()

    def get_label(row):
        key = (row["uid"], row["iid"])
        if key in lookup_uid_iid.index:
            val = lookup_uid_iid.loc[key, "pos_neg_label"]
            if isinstance(val, pd.Series):
                val = val.iloc[0]
            return int(val)
        return 0

    cand_with_tgt["label"] = cand_with_tgt.apply(get_label, axis=1)

    cand_with_tgt = cand_with_tgt.merge(
        tgt[["uid", "iid", "is_target"]],
        on=["uid", "iid"],
        how="left"
    )
    cand_with_tgt["is_target"] = cand_with_tgt["is_target"].fillna(0).astype(int)
    out = cand_with_tgt.merge(user_map, on="uid", how="left").merge(item_map, on="iid", how="left")
    out_csv = BASE / f"candidate_pool_{split}.csv"
    out[["userId", "itemId", "label", "is_target"]].to_csv(out_csv, index=False)
    print(f"Saved candidate_pool_{split}.csv with shape:", out.shape)

for split in ["val", "test"]:
    build_candidate_pool(split)
print("\nDone building candidate_pool_val.csv and candidate_pool_test.csv.")

User map: (14064, 2)
Item map: (18783, 2)
Lookup table: (495535, 6)

--- Building candidate pool for val ---
Loading candidates from: C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\candidates_subset100\val.parquet
Candidates shape (raw): (100, 2)
Candidate columns: ['uid', 'candidates']
Flattened candidates shape: (5199, 2)
Saved candidate_pool_val.csv with shape: (18686, 6)

--- Building candidate pool for test ---
Loading candidates from: C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\candidates_subset100\test.parquet
Candidates shape (raw): (100, 2)
Candidate columns: ['uid', 'candidates']
Flattened candidates shape: (5199, 2)
Saved candidate_pool_test.csv with shape: (18017, 6)

Done building candidate_pool_val.csv and candidate_pool_test.csv.


## 5. Truncate training histories to the last 5 positives per user

To make user histories more comparable to typical sequence lengths and to limit context size for LLM prompts, I keep at most the 5 most recent positive interactions per user in `train_indexed.parquet`.

The original full train file is backed up, and the truncated version overwrites `train_indexed.parquet`.
I also verify minimum and maximum history lengths after truncation to ensure that users still retain enough context for modeling.

In [25]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project")
SPLITS = BASE / "splits"

train_path = SPLITS / "train_indexed.parquet"
train_idx = pd.read_parquet(train_path)

print("Original train_indexed shape:", train_idx.shape)

old_counts = train_idx.groupby("uid")["iid"].size()
print("\n[Old] train positives per user:")
print(old_counts.describe())

def truncate_history(df: pd.DataFrame, k: int = 5) -> pd.DataFrame:
    df_sorted = df.sort_values(["uid", "ts"])
    out = (
        df_sorted
        .groupby("uid", as_index=False)
        .tail(k)
        .sort_values(["uid", "ts"])
        .reset_index(drop=True)
    )
    return out

train_trunc = truncate_history(train_idx, k=5)

new_counts = train_trunc.groupby("uid")["iid"].size()
print("\n[New] train positives per user (after truncation to <=5):")
print(new_counts.describe())

print("\nCheck min/new max history length per user:")
print("  min =", int(new_counts.min()), "  max =", int(new_counts.max()))

backup_path = SPLITS / "train_indexed_full_backup.parquet"
if not backup_path.exists():
    train_idx.to_parquet(backup_path, index=False)
    print("\nBackup saved to:", backup_path)
else:
    print("\nBackup already exists at:", backup_path)

train_trunc.to_parquet(train_path, index=False)
print("\n✔ Overwritten train_indexed.parquet with truncated history.")
print("Final train_indexed shape:", train_trunc.shape)

Original train_indexed shape: (141530, 3)

[Old] train positives per user:
count    14064.000000
mean        10.063282
std         23.043830
min          3.000000
25%          3.000000
50%          5.000000
75%          9.000000
max       1456.000000
Name: iid, dtype: float64

[New] train positives per user (after truncation to <=5):
count    14064.000000
mean         4.304323
std          0.856873
min          3.000000
25%          3.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: iid, dtype: float64

Check min/new max history length per user:
  min = 3   max = 5

Backup saved to: C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\splits\train_indexed_full_backup.parquet

✔ Overwritten train_indexed.parquet with truncated history.
Final train_indexed shape: (60536, 3)


## 6. Sanity checks on 100-user subset and candidate pools

I run a final set of checks on the 100-user subset to ensure:

1. The same 100 users appear consistently in train, validation, and test targets.
2. Per-user interaction counts in the truncated train split look reasonable.
3. Each user has exactly one candidate row in both `val` and `test` candidate files.
4. Validation and test candidate pools are identical for each user (same `uid` order and same candidate lists).

These checks confirm that the subset is internally consistent before exporting it for experiments.

In [26]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project")
SPLITS = BASE / "splits"
CANDS  = BASE / "candidates_subset100"

train_idx = pd.read_parquet(SPLITS / "train_indexed.parquet")
val_idx_sub  = pd.read_parquet(CANDS / "val_targets_indexed.parquet")
test_idx_sub = pd.read_parquet(CANDS / "test_targets_indexed.parquet")

cand_val  = pd.read_parquet(CANDS / "val.parquet")
cand_test = pd.read_parquet(CANDS / "test.parquet")

print("Loaded train_idx:", train_idx.shape)
print("Loaded val_idx_sub:",  val_idx_sub.shape)
print("Loaded test_idx_sub:", test_idx_sub.shape)
print("Loaded cand_val:",  cand_val.shape)
print("Loaded cand_test:", cand_test.shape)

users_train = set(train_idx['uid'].unique())
users_val   = set(val_idx_sub['uid'].unique())
users_test  = set(test_idx_sub['uid'].unique())

subset_users = users_val
print("\n[1] User counts")
print("train users in subset:", len(users_train & subset_users))
print("val users:",  len(users_val))
print("test users:", len(users_test))

train_sub = train_idx[train_idx['uid'].isin(subset_users)]

train_counts = train_sub.groupby("uid")['iid'].nunique()
val_counts   = val_idx_sub.groupby("uid")['iid'].nunique()
test_counts  = test_idx_sub.groupby("uid")['iid'].nunique()

print("\n[2] Per-user counts (subset)")
print("Train counts stats:\n", train_counts.describe())
print("Val counts unique values:",  val_counts.unique())
print("Test counts unique values:", test_counts.unique())

print("\n[3] Candidate rows per user")
print("val candidate users:",  cand_val['uid'].nunique())
print("test candidate users:", cand_test['uid'].nunique())

cand_val_sorted  = cand_val.sort_values("uid").reset_index(drop=True)
cand_test_sorted = cand_test.sort_values("uid").reset_index(drop=True)

same_uid_order = (cand_val_sorted['uid'].tolist() ==
                  cand_test_sorted['uid'].tolist())
same_pools = all(
    list(v) == list(t)
    for v, t in zip(cand_val_sorted['candidates'],
                    cand_test_sorted['candidates'])
)

print("\n[4] Same candidate pool for val & test?")
print("same uid order :", same_uid_order)
print("same pools     :", same_pools)

Loaded train_idx: (60536, 3)
Loaded val_idx_sub: (100, 4)
Loaded test_idx_sub: (100, 4)
Loaded cand_val: (100, 2)
Loaded cand_test: (100, 2)

[1] User counts
train users in subset: 100
val users: 100
test users: 100

[2] Per-user counts (subset)
Train counts stats:
 count    100.000000
mean       4.090000
std        0.911154
min        3.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: iid, dtype: float64
Val counts unique values: [1]
Test counts unique values: [1]

[3] Candidate rows per user
val candidate users: 100
test candidate users: 100

[4] Same candidate pool for val & test?
same uid order : True
same pools     : True


## 7. Export final splits and candidate pools to CSV

Once all checks pass, I export the key Parquet artifacts to CSV under `csv_export/`:

1. `train_indexed.csv`, `val_targets_indexed.csv`, `test_targets_indexed.csv`
2. `user_id_map.csv`, `item_id_map.csv`
3. `candidates_val.csv`, `candidates_test.csv`

These CSV files are the main interface for downstream models, notebooks, and LLM prompting pipelines.

In [27]:
import pandas as pd
from pathlib import Path

BASE = Path("C:/Users/carlk/OneDrive/Documents/uoft/ECE1508H F/Project")
SPLITS = BASE / "splits"
CAND = BASE / "candidates_subset100"

OUT = BASE / "csv_export"
OUT.mkdir(parents=True, exist_ok=True)

print("Loading latest (TRUNCATED) splits...")

train_idx = pd.read_parquet(SPLITS / "train_indexed.parquet")
val_tgt   = pd.read_parquet(SPLITS / "val_targets_indexed.parquet")
test_tgt  = pd.read_parquet(SPLITS / "test_targets_indexed.parquet")

user_map  = pd.read_parquet(SPLITS / "user_id_map.parquet")
item_map  = pd.read_parquet(SPLITS / "item_id_map.parquet")

cand_val  = pd.read_parquet(CAND / "val.parquet")
cand_test = pd.read_parquet(CAND / "test.parquet")

train_idx.to_csv(OUT / "train_indexed.csv", index=False)
val_tgt.to_csv(OUT / "val_targets_indexed.csv", index=False)
test_tgt.to_csv(OUT / "test_targets_indexed.csv", index=False)

user_map.to_csv(OUT / "user_id_map.csv", index=False)
item_map.to_csv(OUT / "item_id_map.csv", index=False)

cand_val.to_csv(OUT / "candidates_val.csv", index=False)
cand_test.to_csv(OUT / "candidates_test.csv", index=False)

print("\n✓ All CSV exported to:", OUT)

Loading latest (TRUNCATED) splits...

✓ All CSV exported to: C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\csv_export


## 8. Build simple item-level metadata (title + one review field)

For a lightweight metadata table, I join the `item_id_map` with the original Amazon CSV to obtain, for each indexed item:

1. the raw string `itemId` from Kaggle,
2. the internal integer `iid`,
3. the book `title`,
4. a single review text field chosen from `review/summary` or `review/text` (whichever is available).

The result is saved as `csv_export/amazonbooks_metadata_with_title_desc.csv` and provides a compact description per item suitable for quick inspection or simple LLM prompts.

In [28]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:/Users/carlk/OneDrive/Documents/uoft/ECE1508H F/Project")

item_map = pd.read_parquet(BASE / "splits" / "item_id_map.parquet")
print("item_map:", item_map.shape)

meta_path = BASE / "Books_rating.csv"
raw = pd.read_csv(meta_path, dtype=str)
print("raw metadata:", raw.shape)

description_col = "review/summary" if "review/summary" in raw.columns else "review/text"

metadata = raw[["Id", "Title", description_col]].copy()
metadata = metadata.rename(columns={
    "Id": "itemId",
    "Title": "title",
    description_col: "description"
})
print("metadata cleaned:", metadata.shape)

merged = item_map.merge(metadata, on="itemId", how="left")
print("Final merged shape:", merged.shape)

out_file = BASE / "csv_export" / "amazonbooks_metadata_with_title_desc.csv"
merged.to_csv(out_file, index=False)

print("\nSaved final metadata to:", out_file)

item_map: (18783, 2)
raw metadata: (3000000, 10)
metadata cleaned: (3000000, 3)
Final merged shape: (1274120, 4)

Saved final metadata to: C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\csv_export\amazonbooks_metadata_with_title_desc.csv


### 8.1 Quick quality checks on metadata

I load `amazonbooks_metadata_with_title_desc.csv` and verify that:

1. All required columns (`itemId`, `iid`, `title`, `description`) are present.
2. `itemId` and `iid` are unique and aligned (no duplicated IDs).
3. Missing titles or descriptions are rare or absent.
4. Sample rows look reasonable and match expectations for well-formed book metadata.

These checks ensure the basic metadata table is clean before building richer descriptions.

In [29]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:/Users/carlk/OneDrive/Documents/uoft/ECE1508H F/Project")
meta_file = BASE / "csv_export" / "amazonbooks_metadata_with_title_desc.csv"

print("Loading:", meta_file)
df = pd.read_csv(meta_file, dtype=str)
print("Loaded shape:", df.shape)
print("Columns:", list(df.columns))

print("\n[1] Check required columns")
for col in ["itemId", "iid", "title", "description"]:
    print(f"  {col}: {'OK' if col in df.columns else 'MISSING'}")

print("\n[2] Uniqueness checks")
print("  unique itemId:", df["itemId"].nunique())
print("  rows (should match if no duplicated itemId):", len(df))
print("  unique iid    :", df["iid"].nunique())

print("\n[3] Missing values")
print("  missing title       :", df["title"].isna().sum())
print("  missing description :", df["description"].isna().sum())

print("\n[4] Sample rows")
print(df.head(10))

Loading: C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\csv_export\amazonbooks_metadata_with_title_desc.csv
Loaded shape: (1274120, 4)
Columns: ['itemId', 'iid', 'title', 'description']

[1] Check required columns
  itemId: OK
  iid: OK
  title: OK
  description: OK

[2] Uniqueness checks
  unique itemId: 18783
  rows (should match if no duplicated itemId): 1274120
  unique iid    : 18783

[3] Missing values
  missing title       : 0
  missing description : 147

[4] Sample rows
       itemId iid                  title  \
0  0001047655   0  The Prodigal Daughter   
1  0001047655   0  The Prodigal Daughter   
2  0001047655   0  The Prodigal Daughter   
3  0001047655   0  The Prodigal Daughter   
4  0001047655   0  The Prodigal Daughter   
5  0001047655   0  The Prodigal Daughter   
6  0001047655   0  The Prodigal Daughter   
7  0001047655   0  The Prodigal Daughter   
8  0001047655   0  The Prodigal Daughter   
9  0001047655   0  The Prodigal Daughter   

                     

### 8.2 Inspect raw Kaggle review columns

I briefly inspect the original `Books_rating.csv` schema to understand which text fields are available (`review/summary`, `review/text`, etc.).
This informs how to combine multiple reviews into a single, cleaned description per item in the next step.

In [30]:
import pandas as pd

raw = pd.read_csv("C:/Users/carlk/OneDrive/Documents/uoft/ECE1508H F/Project/Books_rating.csv")
print(raw.columns)

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')


## 9. Merge multiple reviews into short per-item descriptions

To create richer, LLM-friendly metadata, I aggregate multiple reviews per item into a single cleaned description:

1. Read the raw review text for each `(itemId, Title)` pair from the original CSV.
2. Join with `item_id_map` so every review row has an integer `iid`.
3. For each `(itemId, iid, title)` group, select up to 5 informative reviews.
4. Concatenate them, decode basic HTML entities, and truncate the description to at most 400 characters, cutting at word boundaries when possible.

The final per-item table `(itemId, iid, title, description)` is saved as `metadata/amazonbooks_metadata_merged_per_iid_clean.csv`.

In [31]:
import pandas as pd
from pathlib import Path
from html import unescape

# Paths
RAW_CSV_PATH = BASE / "Books_rating.csv"              # original Kaggle CSV
ITEM_MAP_PATH = BASE / "splits" / "item_id_map.parquet"
OUT_PATH      = BASE / "metadata" / "amazonbooks_metadata_merged_per_iid_clean.csv"

assert RAW_CSV_PATH.exists(),  f"Missing raw CSV: {RAW_CSV_PATH}"
assert ITEM_MAP_PATH.exists(), f"Missing item_id_map: {ITEM_MAP_PATH}"

# Load raw reviews (itemId, title, review text)
raw = pd.read_csv(
    RAW_CSV_PATH,
    usecols=["Id", "Title", "review/text"],
)
raw = raw.rename(columns={"Id": "itemId", "Title": "title", "review/text": "review"})
raw = raw.dropna(subset=["title"])

print("Raw reviews rows:", len(raw))

# Join with item_id_map so every row has an iid
item_map = pd.read_parquet(ITEM_MAP_PATH)   # columns: itemId, iid
df = raw.merge(item_map, on="itemId", how="inner")
print("After join with item_id_map, rows:", len(df))
print("Unique iids in joined reviews:", df["iid"].nunique())


def build_description(group: pd.DataFrame,
                      max_reviews: int = 5,
                      max_chars: int = 400) -> str:
    """Merge up to max_reviews reviews into a short cleaned description."""
    texts = group["review"].dropna().astype(str).head(max_reviews).tolist()
    if not texts:
        return ""

    text = " ".join(texts)
    text = unescape(text)

    if len(text) > max_chars:
        cut = text.rfind(" ", 0, max_chars)
        if cut == -1:
            cut = max_chars
        text = text[:cut]

    return text


# Build one description per iid (and title)
meta = (
    df.groupby(["itemId", "iid", "title"], as_index=False)
      .apply(lambda g: pd.Series({"description": build_description(g)}))
)

print("Metadata rows (one per iid):", len(meta))
print("Unique titles in metadata:", meta["title"].nunique())

# Save final metadata table
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
meta.to_csv(OUT_PATH, index=False, encoding="utf-8")
print(f"Saved merged metadata to {OUT_PATH}")

Raw reviews rows: 2999792
After join with item_id_map, rows: 1274120
Unique iids in joined reviews: 18783
Metadata rows (one per iid): 18783
Unique titles in metadata: 18783
Saved merged metadata to C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\metadata\amazonbooks_metadata_merged_per_iid_clean.csv


C:\Users\carlk\AppData\Local\Temp\ipykernel_14648\2087733464.py:55: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({"description": build_description(g)}))


### 9.1 Final validation of per-item metadata

Finally, I check that the merged metadata file `amazonbooks_metadata_merged_per_iid_clean.csv` satisfies:

1. Each `iid` maps to exactly one `title`.
2. Each `title` maps to exactly one `iid`.
3. The number of rows equals both the number of unique `iid` values and the number of unique titles.

Passing these checks confirms that the cleaned metadata has a strict one-to-one mapping between internal item indices and human-readable book titles, making it safe for downstream models and LLM prompts.

In [32]:
from pathlib import Path
import pandas as pd

# Path to the final per-iid metadata file
META_PATH = BASE / "metadata" / "amazonbooks_metadata_merged_per_iid_clean.csv"

assert META_PATH.exists(), f"Missing metadata file: {META_PATH}"

df = pd.read_csv(META_PATH)

print("Loaded metadata:", META_PATH)
print("Shape:", df.shape)
print("Columns:", list(df.columns))

# Basic uniqueness counts
print("\n[1] Basic uniqueness stats")
n_iid = df["iid"].nunique()
n_title = df["title"].nunique()
n_rows = len(df)

print(" unique iid   :", n_iid)
print(" unique title :", n_title)
print(" rows         :", n_rows)

# Check that each iid has exactly one title
print("\n[2] Check: at most one title per iid")
titles_per_iid = df.groupby("iid")["title"].nunique()
max_titles_per_iid = titles_per_iid.max()
iids_with_multiple_titles = (titles_per_iid > 1).sum()

print(" max titles per iid        :", max_titles_per_iid)
print(" iids with >1 different title:", iids_with_multiple_titles)

# Check that each title maps to exactly one iid
print("\n[3] Check: at most one iid per title")
iids_per_title = df.groupby("title")["iid"].nunique()
max_iids_per_title = iids_per_title.max()
titles_with_multiple_iids = (iids_per_title > 1).sum()

print(" max iids per title        :", max_iids_per_title)
print(" titles with >1 different iid:", titles_with_multiple_iids)

# Final verdict
print("\n[4] Summary")
if max_titles_per_iid == 1 and max_iids_per_title == 1:
    print("OK: Each iid and each title form a one-to-one mapping (titles are unique after processing).")
else:
    print("WARNING: There are still iid<->title conflicts. Please inspect the counts above.")

Loaded metadata: C:\Users\carlk\OneDrive\Documents\uoft\ECE1508H F\Project\metadata\amazonbooks_metadata_merged_per_iid_clean.csv
Shape: (18783, 4)
Columns: ['itemId', 'iid', 'title', 'description']

[1] Basic uniqueness stats
 unique iid   : 18783
 unique title : 18783
 rows         : 18783

[2] Check: at most one title per iid
 max titles per iid        : 1
 iids with >1 different title: 0

[3] Check: at most one iid per title
 max iids per title        : 1
 titles with >1 different iid: 0

[4] Summary
OK: Each iid and each title form a one-to-one mapping (titles are unique after processing).
